In [1]:
%matplotlib inline
%run ../../import_envs.py
%run ../apg_hyperparams.py

CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')

probtorch: 0.0+5a2c637 torch: 1.3.0
inference method:apg, resampling:systematic, apg sweeps:4, epochs:15, sample size:20, batch size:5, learning rate:0.0002


In [6]:
K = 5
T = 20
# train data path
DATA_PATHS = []
TEST_DATA_DIR = '/data/hao/apg_data/bmnist/test_%dT_%dD/' % (T, K)
print('===Loading testing data from %s===' % TEST_DATA_DIR)
# for file in os.listdir(TEST_DATA_DIR):
#     DATA_PATHS.append(os.path.join(TEST_DATA_DIR, file))

DATA_PATHS.append(os.path.join(TEST_DATA_DIR, 'ob-1.npy'))
print('===%d groups of data files are loaded.===' % len(DATA_PATHS))

===Loading testing data from /data/hao/apg_data/bmnist/test_20T_5D/===
===1 groups of data files are loaded.===


In [7]:
from apg_modeling import init_model
from affine_transformer import Affine_Transformer

model, optimizer = init_model(frame_pixels=FRAME_PIXELS,
                               digit_pixels=DIGIT_PIXELS, 
                               num_hidden_digit=NUM_HIDDEN_DIGIT, 
                               num_hidden_coor=NUM_HIDDEN_COOR, 
                               z_where_dim=Z_WHERE_DIM, 
                               z_what_dim=Z_WHAT_DIM, 
                               CUDA=CUDA, 
                               DEVICE=DEVICE, 
                               LOAD_VERSION=LOAD_VERSION, 
                               LR=LR)
AT = Affine_Transformer(frame_pixels=FRAME_PIXELS, 
                        digit_pixels=DIGIT_PIXELS, 
                        CUDA=CUDA, 
                        DEVICE=DEVICE)

In [8]:
## visualize samples
%run ../apg_testing.py
from viz import viz_samples

APG_SWEEPS_TEST = 19
BATCH_SIZE_TEST = 10
SAMPLE_SIZE_TEST = 10
# SAVE_NAME = 'samples-%d' % DATA_PTR
metrics = test_all(model=model,
                   AT=AT, 
                   apg_sweeps=APG_SWEEPS_TEST, 
                   data_paths=DATA_PATHS, 
                   mnist_mean_path=MNIST_MEAN_PATH, 
                   T=T, 
                   K=K, 
                   D=2, 
                   z_what_dim=Z_WHAT_DIM, 
                   batch_size=BATCH_SIZE_TEST, 
                   sample_size=SAMPLE_SIZE_TEST, 
                   CUDA=CUDA, 
                   DEVICE=DEVICE)

1 / 100 in group 0 completed (25s)
2 / 100 in group 0 completed (25s)
3 / 100 in group 0 completed (26s)
4 / 100 in group 0 completed (27s)
5 / 100 in group 0 completed (26s)
6 / 100 in group 0 completed (26s)
7 / 100 in group 0 completed (26s)
8 / 100 in group 0 completed (26s)
9 / 100 in group 0 completed (26s)
10 / 100 in group 0 completed (26s)
11 / 100 in group 0 completed (26s)


In [9]:
mses = []
for mse in metrics['mse']:
    mses.append(mse.unsqueeze(0))
mses = torch.cat(mses, 0)

np.save('mse_%dT_%dD' % (T, K), mses.mean(0).data.numpy())

In [ ]:
torch.Tensor(metrics['mse']).mean()

In [ ]:


metrics = test_single(model=model, 
                      AT=AT,
                      apg_sweeps=APG_SWEEPS_TEST, 
                      frames=frames, 
                      mnist_mean_path=MNIST_MEAN_PATH,
                      K=K,
                      sample_size=10, 
                      CUDA=CUDA, 
                      DEVICE=DEVICE)

viz_samples(frames=frames,
            metrics=metrics,
            apg_sweeps=APG_SWEEPS_TEST,
            K=K, 
            viz_interval=3, 
            figure_size=15, 
            title_fontsize=20, 
            linewidth=2,
            colors=['#AA3377', '#EE7733', '#009988', '#0077BB', '#BBBBBB', '#EE3377', '#DDCC77'],
            save_name=None)

In [ ]:
metrics['density']

In [21]:
# import matplotlib.pyplot as plt
# fs = 8
# COLORS = ['#EE7733', '#009988', '#AA3377', '#0077BB', '#555555', '#999933']
# lw = 3
# fs_title= 14
# fontsize = 20
# fig = plt.figure(figsize=(2.5*fs,fs))
# ax1 = fig.add_subplot(1, 2, 1)
# ax2 = fig.add_subplot(1, 2, 2)
# Ts = [20, 50, 100]
# for i, t in enumerate(Ts):
#     mse = np.load('mse_%dT_3D.npy' % (t))
#     print(mse[11])
#     ax1.plot(mse, 'o-', c=COLORS[i], linewidth=lw, label='T=%d' % t)
# for i, t in enumerate(Ts):
#     mse = np.load('mse_%dT_5D.npy' % (t))
#     ax2.plot(mse, 'o-', c=COLORS[i+3], linewidth=lw, label='T=%d' % t)
#     print(mse[11])
# ax1.legend(fontsize=fontsize)
# ax2.legend(fontsize=fontsize)